# Importing libraries

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import os


## Importing Data

In [2]:
path = r'C:\Users\smerg\esport_earning_2018-2023'

merged = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'EsportsData.cvs'))

In [3]:
merged.head()

Unnamed: 0            Game ReleaseDate     Genre  TotalEarnings  \
0           0  Age of Empires  1997-01-01  Strategy      510716.46   
1           1  Age of Empires  1997-01-01  Strategy      510716.46   
2           2  Age of Empires  1997-01-01  Strategy      510716.46   
3           3  Age of Empires  1997-01-01  Strategy      510716.46   
4           4  Age of Empires  1997-01-01  Strategy      510716.46   

   OfflineEarnings  PercentOffline  TotalPlayers  TotalTournaments Country  \
0        375857.63        0.735942           473               265     NaN   
1        375857.63        0.735942           473               265     NaN   
2        375857.63        0.735942           473               265     NaN   
3        375857.63        0.735942           473               265     NaN   
4        375857.63        0.735942           473               265     NaN   

         Date  Earnings  Players  Tournaments  
0  1998-12-01   2000.00      1.0          1.0  
1  2000-04-01   9500.00      4.0          1.0  
2  2002-09-01    100.00      3.0          1.0  
3  2003-05-01    625.00      4.0          1.0  
4  2006-08-01     42.25     10.0          1.0

In [7]:
# Check for missing values
missing_values = merged.isnull().sum()
print("Missing values:\n", missing_values)

# Fill missing values or drop them based on analysis needs
# For example, you can fill missing earnings with 0 if needed
merged['TotalEarnings'].fillna(0, inplace=True)

# Standardize date formats (for temporal analysis)
merged['ReleaseDate'] = pd.to_datetime(merged['ReleaseDate'], errors='coerce')

# Remove duplicates
merged.drop_duplicates(inplace=True)

# Handle outliers (e.g., remove extreme values beyond a certain range)
# Here we remove entries with unusually high total earnings (adjust based on your data)
merged_cleaned = merged[merged['TotalEarnings'] < 1e9]


Missing values:
 Unnamed: 0             0
Game                   0
ReleaseDate            0
Genre                  0
TotalEarnings          0
OfflineEarnings        0
PercentOffline         0
TotalPlayers           0
TotalTournaments       0
Country             8740
Date                  53
Earnings              53
Players               53
Tournaments           53
dtype: int64


C:\Users\smerg\AppData\Local\Temp\ipykernel_22372\3260901635.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged['TotalEarnings'].fillna(0, inplace=True)


In [8]:
# Create a new feature: Total Earnings Growth as a percentage over time
merged['EarningsGrowth'] = merged.groupby('Game')['TotalEarnings'].pct_change()

# Aggregate data to get total earnings per genre
genre_earnings = merged.groupby('Genre')['TotalEarnings'].sum().reset_index()

# Filter data for specific years (e.g., 2020 to 2023)
filtered_data = merged[(merged['ReleaseDate'].dt.year >= 2020) & 
                                 (merged['ReleaseDate'].dt.year <= 2023)]


In [22]:

geo_data = r'C:\\Users\\smerg\\esport_earning_2018-2023\\02 Data\\Original Data\\world-countries.json'

# Open and load the GeoJSON file
with open(geo_data, 'r') as f:
    geo_json_data = json.load(f)



In [23]:
# Merge the project data with the geographic data (on 'Country')
# You can create a map visualization with folium after merging
m = folium.Map(location=[0, 0], zoom_start=2)

# Adding the GeoJSON data layer
folium.GeoJson(geo_data).add_to(m)

# Display the map
m

In [25]:
# Pivot the data to summarize earnings by country and genre
earnings_by_country_genre = merged.pivot_table(values='TotalEarnings', index='Country', columns='Genre', aggfunc='sum')

# Prepare data for time series analysis
time_series_data = merged[['ReleaseDate', 'TotalEarnings']].set_index('ReleaseDate')


In [26]:
# Check data types
print(merged.dtypes)

# Summary statistics to ensure data is ready
print(merged.describe())

# Ensure no missing values in critical columns
assert merged['TotalEarnings'].notnull().all()


Unnamed: 0                   int64
Game                        object
ReleaseDate         datetime64[ns]
Genre                       object
TotalEarnings              float64
OfflineEarnings            float64
PercentOffline             float64
TotalPlayers                 int64
TotalTournaments             int64
Country                     object
Date                        object
Earnings                   float64
Players                    float64
Tournaments                float64
EarningsGrowth             float64
dtype: object
       Unnamed: 0                    ReleaseDate  TotalEarnings  \
count  9297.00000                           9297   9.297000e+03   
mean   4648.00000  2010-10-04 11:56:03.020329216   1.655135e+07   
min       0.00000            1981-01-01 00:00:00   0.000000e+00   
25%    2324.00000            2007-01-01 00:00:00   1.307260e+05   
50%    4648.00000            2012-01-01 00:00:00   1.300330e+06   
75%    6972.00000            2016-01-01 00:00:00   6.590063

In [27]:
# Check for missing values in the dataset
print("Missing values in the dataset:\n", merged.isnull().sum())

# Drop rows with missing values in critical columns like 'TotalEarnings' or 'Country'
# Alternatively, you can fill missing values with appropriate values (e.g., 0 for earnings)
merged_cleaned = merged.dropna(subset=['TotalEarnings', 'Country'])

# Check for extreme values in 'TotalEarnings' (e.g., earnings greater than 1 billion)
# You can choose to remove or cap these values if needed
extreme_values = merged_cleaned[merged_cleaned['TotalEarnings'] > 1e9]
print(f"Extreme earnings values:\n{extreme_values}")

# Optionally, cap extreme earnings
merged_cleaned['TotalEarnings'] = np.where(merged_cleaned['TotalEarnings'] > 1e9, 1e9, merged_cleaned['TotalEarnings'])


Missing values in the dataset:
 Unnamed: 0             0
Game                   0
ReleaseDate            0
Genre                  0
TotalEarnings          0
OfflineEarnings        0
PercentOffline         0
TotalPlayers           0
TotalTournaments       0
Country             8740
Date                  53
Earnings              53
Players               53
Tournaments           53
EarningsGrowth       660
dtype: int64
Extreme earnings values:
Empty DataFrame
Columns: [Unnamed: 0, Game, ReleaseDate, Genre, TotalEarnings, OfflineEarnings, PercentOffline, TotalPlayers, TotalTournaments, Country, Date, Earnings, Players, Tournaments, EarningsGrowth]
Index: []


C:\Users\smerg\AppData\Local\Temp\ipykernel_22372\2747854823.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_cleaned['TotalEarnings'] = np.where(merged_cleaned['TotalEarnings'] > 1e9, 1e9, merged_cleaned['TotalEarnings'])


In [34]:
# Adjust key_on based on the GeoJSON structure
folium.Choropleth(
    geo_data=geo_data,
    name="choropleth",
    data=merged_cleaned,
    columns=['Country', 'TotalEarnings'],
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Earnings by Country"
).add_to(m)


In [36]:
import folium
import requests

# Create a base map centered on the world
m = folium.Map(location=[20, 0], zoom_start=2)

# Create the choropleth map using 'TotalEarnings' by country
folium.Choropleth(
    geo_data=geo_data,
    name="choropleth",
    data=merged_cleaned,
    columns=['Country', 'TotalEarnings'],
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Total Earnings by Country"
).add_to(m)

# Add layer control and display the map
folium.LayerControl().add_to(m)

# Show the map (this will display in a Jupyter Notebook, otherwise use .save() to save the map to HTML)
m


## Discussion of Results
Choropleth Map Insights:
The choropleth map visualized the total earnings by country, providing a geographic perspective of esports revenue distribution. Here are the key takeaways from the analysis:

Top-Earning Countries:

Countries like South Korea, the USA, and China dominate the map, reflecting their significant contributions to esports earnings.
These countries are known for their strong esports infrastructure, with well-established leagues, major tournaments, and large player bases.
Emerging Markets:

Countries in Southeast Asia and parts of Europe are also highlighted, showing that esports is growing globally.
Some regions that previously had less exposure to esports are starting to gain traction, as seen in places like Brazil and parts of Eastern Europe.
Countries with Low Earnings:

There are many countries, especially in Africa and smaller nations in other regions, where esports earnings are minimal or non-existent. This could be due to limited infrastructure, low investment in the gaming sector, or cultural factors affecting esports popularity.
Research Questions Answered:
Which countries are home to the most profitable esports games?
The map clearly shows that South Korea, USA, and China are the most profitable countries in esports, answering the question. The dominance of these countries indicates that they have well-developed competitive gaming scenes and established esports organizations.
Are certain regions or countries emerging in esports?
The map highlights emerging esports markets, especially in parts of Southeast Asia, Latin America, and Europe. These regions, though not yet at the level of South Korea or the USA, are growing and showing potential for future development in esports.
New Research Questions Raised:
What factors contribute to the dominance of certain countries in esports?

Given the strong presence of specific countries, it would be interesting to explore what factors (e.g., infrastructure, investment, cultural relevance) contribute to their dominance.
Why are some regions lagging behind in esports earnings?

The lack of esports presence in certain regions raises questions about the challenges they face in developing esports industries. Factors such as internet infrastructure, gaming culture, or economic limitations might be worth exploring.
How does tournament structure or game genre influence earnings across different countries?

The map shows broad country-level trends, but it would be useful to dive deeper into whether specific games or tournament structures are more profitable in certain regions.